In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta

# Standard plotly imports
import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
# Using plotly + cufflinks in offline mode
import cufflinks
cufflinks.go_offline(connected=True)
init_notebook_mode(connected=True)

In [2]:
data = pd.read_csv('dashboard_data.csv')
df = data.groupby(['date', 'Country/Region'], as_index=False)['Confirmed'].sum()
df['previous_week'] = df.groupby(['Country/Region'])['Confirmed'].shift(7, fill_value=0)
df['new_cases'] = df['Confirmed'] - df['previous_week']

In [3]:
threshold = 50
countries = df.groupby(by='Country/Region', as_index=False)['Confirmed'].max().sort_values(by='Confirmed', ascending=False)
countries = countries[countries['Confirmed'] > threshold]['Country/Region'].unique()

fig = go.Figure()
for country in countries:
    filtered_df = df[df['Country/Region'] == country].reset_index()
    idx = filtered_df['Confirmed'].sub(threshold).gt(0).idxmax()
    trace_data = filtered_df[idx:]
    fig.add_trace(go.Scatter(x=trace_data['Confirmed'],
                             y=trace_data['new_cases'],
                             mode='lines',
                             name=country,
                             hoverinfo='x+name'))
fig.update_layout(xaxis_type="log",
                  yaxis_type="log",
                  title='Trajectory of COVID-19 Confirmed Cases (countries with greater than 50 confirmed cases)',
                  xaxis_title='Total Confirmed Cases',
                  yaxis_title='New Confirmed Cases (in the past week)')
fig.show()

In [4]:
df2 = data[data['Country/Region'] == 'US'].groupby(['date', 'Province/State'], as_index=False)['Confirmed'].sum()
df2['previous_week'] = df2.groupby(['Province/State'])['Confirmed'].shift(7, fill_value=0)
df2['new_cases'] = df2['Confirmed'] - df2['previous_week']

In [5]:
threshold = 50
states = df2.groupby(by='Province/State', as_index=False)['Confirmed'].max().sort_values(by='Confirmed', ascending=False)
states = states[states['Confirmed'] > threshold]['Province/State'].unique()

fig = go.Figure()
for state in states:
    filtered_df = df2[df2['Province/State'] == state].reset_index()
    idx = filtered_df['Confirmed'].sub(threshold).gt(0).idxmax()
    trace_data = filtered_df[idx:]
    fig.add_trace(go.Scatter(x=trace_data['Confirmed'],
                             y=trace_data['new_cases'],
                             mode='lines',
                             name=state,
                             hoverinfo='x+name'))
fig.update_layout(xaxis_type="log",
                  yaxis_type="log",
                  title='Trajectory of COVID-19 Confirmed Cases (states with greater than 50 confirmed cases)',
                  xaxis_title='Total Confirmed Cases',
                  yaxis_title='New Confirmed Cases (in the past week)')
fig.show()